In [1]:
!pip install -r requirements.txt

     ---------------------------------------- 0.0/803.1 kB ? eta -:--:--
     ------------- ------------------------ 286.7/803.1 kB 8.6 MB/s eta 0:00:01
     ----------------------------------- -- 747.5/803.1 kB 9.3 MB/s eta 0:00:01
     -------------------------------------- 803.1/803.1 kB 8.4 MB/s eta 0:00:00
     ---------------------------------------- 0.0/146.9 kB ? eta -:--:--
     ---------------------------------------- 146.9/146.9 kB ? eta 0:00:00
     ---------------------------------------- 0.0/635.3 kB ? eta -:--:--
     ------------------------------- ----- 542.7/635.3 kB 11.3 MB/s eta 0:00:01
     ------------------------------------- 635.3/635.3 kB 13.3 MB/s eta 0:00:00
     ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
     - -------------------------------------- 0.4/10.8 MB 13.2 MB/s eta 0:00:01
     --- ------------------------------------ 0.9/10.8 MB 11.9 MB/s eta 0:00:01
     ----- ---------------------------------- 1.5/10.8 MB 11.6 MB/s eta 0:

In [2]:
from dotenv import load_dotenv
load_dotenv()


True

In [3]:
from langchain.utilities import SQLDatabase

In [4]:
db_user = "root"
db_password = "mysql2023"
db_host = "localhost"
db_name= "atliq_tshirts"

In [5]:
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=3)

In [42]:
print(db.table_info)


CREATE TABLE discounts (
	discount_id INTEGER NOT NULL AUTO_INCREMENT, 
	t_shirt_id INTEGER NOT NULL, 
	pct_discount DECIMAL(5, 2), 
	PRIMARY KEY (discount_id), 
	CONSTRAINT discounts_ibfk_1 FOREIGN KEY(t_shirt_id) REFERENCES t_shirts (t_shirt_id), 
	CONSTRAINT discounts_chk_1 CHECK ((`pct_discount` between 0 and 100))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from discounts table:
discount_id	t_shirt_id	pct_discount
100	1	10.00
200	2	15.00
500	5	25.00
*/


CREATE TABLE t_shirts (
	t_shirt_id INTEGER NOT NULL AUTO_INCREMENT, 
	brand ENUM('Van Huesen','Levi','Nike','Adidas') NOT NULL, 
	color ENUM('Red','Blue','Black','White') NOT NULL, 
	size ENUM('XS','S','M','L','XL') NOT NULL, 
	price INTEGER, 
	stock_quantity INTEGER NOT NULL, 
	PRIMARY KEY (t_shirt_id), 
	CONSTRAINT t_shirts_chk_1 CHECK ((`price` between 10 and 50))
)DEFAULT CHARSET=utf8mb4 COLLATE utf8mb4_0900_ai_ci ENGINE=InnoDB

/*
3 rows from t_shirts table:
t_shirt_id	brand	color	size	price

In [6]:
from langchain_experimental.sql import SQLDatabaseChain

In [ ]:
 !pip install langchain-google-genai

In [8]:
from langchain_google_genai import GoogleGenerativeAI

c:\Users\jagar\anaconda3\envs\speech\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from getpass import getpass

api_key = getpass()

In [12]:
llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=api_key)
print(
    llm.invoke(
        "What are some of the pros and cons of Python as a programming language?"
    )
)

**Pros of Python:**

* **Easy to learn:** Python is a very easy-to-learn programming language, even for beginners. It has a simple syntax and is very readable, making it a great choice for people who are new to programming.
* **Versatile:** Python is a very versatile language that can be used for a wide variety of tasks, including web development, data science, and machine learning. It is also supported by a large community of developers, which means that there are plenty of resources available to help you learn and use the language.
* **High-performance:** Python is a fast-performing language, especially when compared to other interpreted languages. This makes it a good choice for developing high-performance applications.
* **Open-source:** Python is an open-source language, which means that it is free to use and modify. This allows you to use the language however you want, without having to worry about licensing fees or restrictions.

**Cons of Python:**

* **Slow:** Python is a slow

In [13]:
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
qns1 = db_chain("How many t-shirts do we have left for nike in extra small size and white color?")



> Entering new SQLDatabaseChain chain...
How many t-shirts do we have left for nike in extra small size and white color?
SQLQuery:SELECT stock_quantity FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'
SQLResult: 
Answer:51
> Finished chain.


In [14]:
qns2 = db_chain.run("SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLQuery:SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'
SQLResult: [(Decimal('16173'),)]
Answer:16173
> Finished chain.


In [15]:
sql_code = """
select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """

qns3 = db_chain.run(sql_code)



> Entering new SQLDatabaseChain chain...

select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 
SQLQuery:select sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
SQLResult: [(Decimal('20140.200000'),)]
Answer:20140.2
> Finished chain.


In [16]:
qns4 = db_chain.run("SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'")



> Entering new SQLDatabaseChain chain...
SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLQuery:SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('21414'),)]
Answer:21414
> Finished chain.


In [17]:
qns5 = db_chain.run("SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'")



> Entering new SQLDatabaseChain chain...
SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLQuery:SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'
SQLResult: [(Decimal('226'),)]
Answer:226
> Finished chain.


In [32]:
few_shots = [
    {'Question' : "How many t-shirts do we have left for Nike in XS size and white color?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'",
     'SQLResult': "Result of the SQL query",
     'Answer' : "51"},
    {'Question': "How much is the total price of the inventory for all S-size t-shirts?",
     'SQLQuery':"SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'",
     'SQLResult': "Result of the SQL query",
     'Answer': qns2},
    {'Question': "If we have to sell all the Levis T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?" ,
     'SQLQuery' : """SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from
(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'
group by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id
 """,
     'SQLResult': "Result of the SQL query",
     'Answer': qns3} ,
     {'Question' : "If we have to sell all the Levis T-shirts today. How much revenue our store will generate without discount?" ,
      'SQLQuery': "SELECT SUM(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'",
      'SQLResult': "Result of the SQL query",
      'Answer' : qns4},
    {'Question': "How many white color Levis shirt I have?",
     'SQLQuery' : "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Levi' AND color = 'White'",
     'SQLResult': "Result of the SQL query",
     'Answer' : qns5
     }
]

In [19]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [20]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello, world!")
vector[:5]

[0.05636945, 0.0048285457, -0.0762591, -0.023642512, 0.05329321]

In [33]:
for example in few_shots:
    print(example.values())

dict_values(['How many t-shirts do we have left for Nike in XS size and white color?', "SELECT sum(stock_quantity) FROM t_shirts WHERE brand = 'Nike' AND color = 'White' AND size = 'XS'", 'Result of the SQL query', '51'])
dict_values(['How much is the total price of the inventory for all S-size t-shirts?', "SELECT SUM(price*stock_quantity) FROM t_shirts WHERE size = 'S'", 'Result of the SQL query', '16173'])
dict_values(['If we have to sell all the Levis T-shirts today with discounts applied. How much revenue  our store will generate (post discounts)?', "SELECT sum(a.total_amount * ((100-COALESCE(discounts.pct_discount,0))/100)) as total_revenue from\n(select sum(price*stock_quantity) as total_amount, t_shirt_id from t_shirts where brand = 'Levi'\ngroup by t_shirt_id) a left join discounts on a.t_shirt_id = discounts.t_shirt_id\n ", 'Result of the SQL query', '20140.2'])
dict_values(['If we have to sell all the Levis T-shirts today. How much revenue our store will generate without disc

In [34]:
to_vectorize = [" ".join(example.values()) for example in few_shots]

In [35]:
from langchain.vectorstores import Chroma

In [36]:
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=few_shots)

In [37]:
from langchain.prompts import SemanticSimilarityExampleSelector

In [38]:
ex_selector=SemanticSimilarityExampleSelector(vectorstore=vectorstore,k=2)

In [39]:
from langchain.chains.sql_database.prompt import PROMPT_SUFFIX,_mysql_prompt

In [40]:
print(_mysql_prompt)

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most {top_k} results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of

In [41]:
print(PROMPT_SUFFIX)

Only use the following tables:
{table_info}

Question: {input}


In [42]:
from langchain.prompts.prompt import PromptTemplate

In [43]:
example_prompt = PromptTemplate(
    input_variables=["Question", "SQLQuery", "SQLResult","Answer",],
    template="\nQuestion: {Question}\nSQLQuery: {SQLQuery}\nSQLResult: {SQLResult}\nAnswer: {Answer}",
)


In [45]:
from langchain.prompts import FewShotPromptTemplate

In [46]:
few_shot_prompt = FewShotPromptTemplate(
    example_selector=ex_selector,
    example_prompt=example_prompt,
    prefix=_mysql_prompt,
    suffix=PROMPT_SUFFIX,
    input_variables=["input", "table_info", "top_k"], #These variables are used in the prefix and suffix
)

In [48]:
new_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True, prompt=few_shot_prompt)

In [50]:
new_chain("If we have to sell all the Levis T-shirts today. How much revenue our store will generate without discount?")



> Entering new SQLDatabaseChain chain...
If we have to sell all the Levis T-shirts today. How much revenue our store will generate without discount?
SQLQuery:SELECT sum(price * stock_quantity) FROM t_shirts WHERE brand = 'Levi'
SQLResult: [(Decimal('21414'),)]
Answer:21414
> Finished chain.


{'query': 'If we have to sell all the Levis T-shirts today. How much revenue our store will generate without discount?',
 'result': '21414'}